In [1]:
#imports:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
pd.set_option("display.max_columns", 25)

In [4]:
pd.set_option("display.max_rows", 50)

In [5]:
test = pd.read_csv('datasets/test.csv')

In [6]:
test.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,...,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,60,112,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,AllPub,Inside,...,24,0,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,184,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,AllPub,Inside,...,76,0,0,185,0,NaN,NaN,NaN,0,7,2009,WD


In [7]:
test['Lot Frontage'] = test['Lot Frontage'].fillna(test['Lot Frontage'].mean())

#dropping columns due to many missing values
#test.drop(columns = ['Alley', 'Fireplace Qu', 'Pool QC', 'Fence', 'Misc Feature'], inplace = True)

test['Bsmt Qual'] = test['Bsmt Qual'].fillna(test['Bsmt Qual'].mode()[0])
test['Bsmt Cond'] = test['Bsmt Cond'].fillna(test['Bsmt Cond'].mode()[0])

test['Garage Type'] = test['Garage Type'].fillna(test['Garage Type'].mode()[0])
test['Garage Yr Blt'] = test['Garage Yr Blt'].fillna(test['Garage Yr Blt'].mode()[0])

test['Garage Finish'] = test['Garage Finish'].fillna(test['Garage Finish'].mode()[0])
test['Garage Qual'] = test['Garage Qual'].fillna(test['Garage Qual'].mode()[0])
test['Garage Cond'] = test['Garage Cond'].fillna(test['Garage Cond'].mode()[0])

test['Mas Vnr Type'] = test['Mas Vnr Type'].fillna(test['Mas Vnr Type'].mode()[0])
test['Mas Vnr Area'] = test['Mas Vnr Area'].fillna(test['Mas Vnr Area'].mode()[0])

test = test.replace(np.nan, 0)

In [8]:
test.shape

(878, 80)

In [9]:
test.isnull().sum().sum()

0

In [10]:
test["Bldg Type"] = test["Bldg Type"].replace(to_replace="2fmCon", value="2FmCon")
test["Bldg Type"] = test["Bldg Type"].replace(to_replace="Duplex", value="Duplx")
test["Bldg Type"] = test["Bldg Type"].replace(to_replace="Twnhs", value="TwnhsI")
test["Exterior 2nd"] = test["Exterior 2nd"].replace(to_replace="Brk Cmn", value="BrkComm")
test["Exterior 2nd"] = test["Exterior 2nd"].replace(to_replace="CmentBd", value="CemntBd")
test["Exterior 2nd"] = test["Exterior 2nd"].replace(to_replace="Wd Shng", value="WdShing")
test["MS Zoning"] = test["MS Zoning"].replace(to_replace="A (agr)", value="A")
test["MS Zoning"] = test["MS Zoning"].replace(to_replace="C (all)", value="C")
test["MS Zoning"] = test["MS Zoning"].replace(to_replace="I (all)", value="I")
test["Neighborhood"] = test["Neighborhood"].replace(to_replace="NAmes", value="Names")
test["Sale Type"] = test["Sale Type"].replace(to_replace="WD ", value="WD")

In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Data columns (total 80 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               878 non-null    int64  
 1   PID              878 non-null    int64  
 2   MS SubClass      878 non-null    int64  
 3   MS Zoning        878 non-null    object 
 4   Lot Frontage     878 non-null    float64
 5   Lot Area         878 non-null    int64  
 6   Street           878 non-null    object 
 7   Alley            878 non-null    object 
 8   Lot Shape        878 non-null    object 
 9   Land Contour     878 non-null    object 
 10  Utilities        878 non-null    object 
 11  Lot Config       878 non-null    object 
 12  Land Slope       878 non-null    object 
 13  Neighborhood     878 non-null    object 
 14  Condition 1      878 non-null    object 
 15  Condition 2      878 non-null    object 
 16  Bldg Type        878 non-null    object 
 17  House Style     

In [12]:
test.describe()

,Id,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,...,Garage Yr Blt,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold
count,878.000000,8.780000e+02,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000,...,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000
mean,1444.749431,7.162867e+08,58.268793,69.545961,10307.033030,6.050114,5.566059,1970.492027,1984.417995,106.115034,441.328018,53.896355,...,1978.207289,1.742597,470.362187,93.430524,47.070615,24.064920,2.597950,14.830296,1.884966,48.498861,6.202733,2007.825740
std,850.838263,1.889103e+08,42.235407,21.279162,10002.674602,1.369065,1.128903,30.395895,20.450725,187.086530,438.457329,178.632355,...,25.786119,0.750391,212.734075,121.181702,68.180939,73.249463,24.962482,53.003794,29.916672,550.169317,2.642498,1.327861
min,2.000000,5.263021e+08,20.000000,21.000000,1477.000000,2.000000,1.000000,1880.000000,1950.000000,0.000000,0.000000,0.000000,...,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,692.250000,5.284841e+08,20.000000,60.000000,7297.250000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,...,1960.000000,1.000000,322.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000
50%,1433.000000,5.354542e+08,50.000000,69.545961,9446.000000,6.000000,5.000000,1972.000000,1992.000000,0.000000,373.000000,0.000000,...,1980.000000,2.000000,473.000000,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,2197.500000,9.071904e+08,70.000000,78.000000,11589.000000,7.000000,6.000000,1999.750000,2003.000000,171.750000,734.750000,0.000000,...,2003.000000,2.000000,576.000000,170.750000,69.750000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,2928.000000,1.007100e+09,190.000000,182.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1378.000000,2288.000000,1526.000000,...,2010.000000,4.000000,1488.000000,690.000000,742.000000,1012.000000,360.000000,576.000000,555.000000,15500.000000,12.000000,2010.000000


In [13]:
test.shape

(878, 80)

In [14]:
numerical_columns = test[['Lot Area', 'Overall Qual', 'Overall Cond', 'Mas Vnr Area', 'Total Bsmt SF', 'Gr Liv Area', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd', 'Fireplaces', 'Garage Cars', 'Garage Area', 'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch', '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Misc Val', 'Mo Sold']]

In [15]:
exter_quality_dummies = pd.get_dummies(test['Exter Qual'], prefix = 'Exter Qual')

In [16]:
kitchen_quality_dummies = pd.get_dummies(test['Kitchen Qual'], prefix = 'Kitchen Qual')

In [17]:
bsmt_quality_dummies = pd.get_dummies(test['Bsmt Qual'], prefix = 'Bsmt Qual')

In [18]:
garage_finish_dummies = pd.get_dummies(test['Garage Finish'], prefix = 'Garage Finish')

In [19]:
house_style_dummies = pd.get_dummies(test['House Style'], prefix = 'House Style')

In [20]:
test = pd.concat([test, bsmt_quality_dummies, exter_quality_dummies, kitchen_quality_dummies, house_style_dummies, garage_finish_dummies], axis = 1)

In [21]:
test.shape

(878, 105)

In [22]:
test.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,...,Kitchen Qual_TA,House Style_1.5Fin,House Style_1.5Unf,House Style_1Story,House Style_2.5Fin,House Style_2.5Unf,House Style_2Story,House Style_SFoyer,House Style_SLvl,Garage Finish_Fin,Garage Finish_RFn,Garage Finish_Unf
0,2658,902301120,190,RM,69.000000,9142,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,0,1,0,0,0,0,1
1,2718,905108090,90,RL,69.545961,9662,Pave,0,IR1,Lvl,AllPub,Inside,...,1,0,0,1,0,0,0,0,0,1,0,0
2,2414,528218130,60,RL,58.000000,17104,Pave,0,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,0,1,0,0,0,1,0
3,1989,902207150,30,RM,60.000000,8520,Pave,0,Reg,Lvl,AllPub,Inside,...,1,0,0,1,0,0,0,0,0,0,0,1
4,625,535105100,20,RL,69.545961,9500,Pave,0,IR1,Lvl,AllPub,Inside,...,1,0,0,1,0,0,0,0,0,0,1,0


In [23]:
test.to_csv("datasets/test_clean20.csv")